In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-07-20"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
8509,2024-07-20,Europa Eurobasket Sub,15:30,Alemanha U20,Israel U20,1.79,1.93,149.5,1.80,1.88,-4.5,2.05,2.03,AywT4DMF,0.558659,0.518135,0.555556,0.531915,0.076794,228.160,189.720409,0.831524,1.8,1.643168,0.912871,88.58,3.006,2.480691,0.825247,10.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,86,113,1.03,2.37,0.000,0.000,0.053223,0.030744,0.006932,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8510,2024-07-20,Europa Eurobasket Sub,13:00,Sérvia U20,Itália U20,1.43,2.68,152.5,1.83,1.87,0.0,0.00,0.00,dW753Ywm,0.699301,0.373134,0.546448,0.534759,0.072435,137.038,93.917882,0.685342,1.8,1.643168,0.912871,102.01,1.710,1.280566,0.748869,49.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,101,63,1.01,1.05,0.000,0.000,0.430114,0.015289,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8511,2024-07-20,Europa Eurobasket Sub,10:30,Montenegro U20,Turquia U20,9.70,1.01,146.5,1.88,1.79,0.0,0.00,0.00,CbIJrT4K,0.103093,0.990099,0.531915,0.558659,0.093192,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,58,81,11.00,1.54,0.000,0.000,1.147480,0.034681,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8512,2024-07-20,Europa Eurobasket Sub,08:00,Islândia U20,Macedônia do Norte U20,1.21,4.13,151.5,1.85,1.85,0.0,0.00,0.00,4xekMd2d,0.826446,0.242131,0.540541,0.540541,0.068577,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,79,59,12.20,9.70,0.000,0.000,0.773315,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8513,2024-07-20,Europa Eurobasket Sub,10:30,Republica Checa U20,Polónia U20,2.76,1.41,144.5,1.85,1.85,0.0,0.00,0.00,dCisOIXq,0.362319,0.709220,0.540541,0.540541,0.071539,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,516.530,106.940859,0.207037,1.2,1.643168,1.369306,426.96,7.610,1.978371,0.259970,-67.0,68,72,2.76,5.93,0.000,0.000,0.457839,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8571,2024-07-20,Austrália Nbl1 Central,07:00,Sturt Sabres,Forestville Eagles,1.89,1.83,187.5,1.87,1.83,-1.5,2.02,1.94,OC4LTNnJ,0.529101,0.546448,0.534759,0.546448,0.075549,177.868,100.963702,0.567633,2.4,1.341641,0.559017,132.50,1.770,0.837228,0.473010,21.0,128.298,24.105382,0.187886,1.8,1.643168,0.912871,151.34,1.276,0.223786,0.175381,62.0,106,94,1.25,1.61,119.808,136.754,0.022810,0.015289,0.028570,-0.24,-0.048,-18.541667,0.000000,0.0,0.000000,1.73,0.346,2.398844,0.000000,0.0,0.000000
8572,2024-07-20,Austrália Nbl1 Sul,06:00,Knox F,Mt. Gambier F,1.19,4.37,148.5,1.80,1.88,0.0,0.00,0.00,vstm9j6S,0.840336,0.228833,0.555556,0.531915,0.069169,135.818,53.229083,0.391915,3.0,0.000000,0.000000,124.83,1.498,0.395942,0.264314,136.0,86.626,14.621389,0.1687

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
8538,16:00,Eua Nba,Sacramento Kings,Phoenix Suns,1.94,Back Home
8558,04:00,Austrália Nbl1 Sul,Sandringham F,Waverley F,1.78,Back Home
